#### Llama2 7B torch.float16 추론 
> * Llama2 모델은 GPU T4x16G 에서는 사용할 수 없다. 공식문서에서는 T4x27G 이상 사용을 권하고 있다.
```
1. casllm-base-7b-hf: Meta Llama2 7B
2. parameter data type: touch.float16
3. qutanization: not quantized
4. processor: cpu
5. write type: safetensors
6. transformers: AutoModelForCausalLM, pipeline
```

In [ ]:
import torch
from transformers import AutoModelForCausalLM, pipeline

In [ ]:
# KoAlpaca-ployglot 12.8B (Non 양자화 모델)
# 파일형식: Hugging Transformers 의 Safe serialization 모델
# 파일크기: 1G X 28개
model_id = "/data/bwllm/models/casllm-base-7b-sf"
model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.float16, device_map="auto")

In [ ]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

In [ ]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

In [ ]:
"""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

# 장치 확인   
print(device)

# 모델을 해당 디바이스로 이동
model.to(device)
"""

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

In [ ]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [ ]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [ ]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템 감리 절차 과정에 대해 설명해줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

In [ ]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)